In [1]:
%matplotlib inline
from directSolve import DS
import numpy as np, matplotlib.pyplot as plt
import pystokes
import matelms as me

In [2]:
def rotation(vector,alpha, beta, gamma):
    rotmat = np.array([[np.cos(beta)*np.cos(gamma), np.sin(alpha)*np.sin(beta)*np.cos(gamma) 
                        - np.cos(alpha)*np.sin(gamma), np.cos(alpha)*np.sin(beta)*np.cos(gamma)
                        + np.sin(alpha)*np.sin(gamma)],
                       [np.cos(beta)*np.sin(gamma),
                        np.sin(alpha)*np.sin(beta)*np.sin(gamma) + np.cos(alpha)*np.cos(gamma),
                        np.cos(alpha)*np.sin(beta)*np.sin(gamma) - np.sin(alpha)*np.cos(gamma)],
                       [-np.sin(beta),
                        np.sin(alpha)*np.cos(beta),
                        np.cos(alpha)*np.cos(beta)]])
    return np.dot(rotmat, vector)

___________________

Try different simulator in case it's actually the ODE solver that goes wrong:

In [3]:
# particle radius, self-propulsion speed, number and fluid viscosity
b, Np, eta = 1.0, 2, 0.1
ds = DS(b, Np, eta)

## squirmer
S0, D0  = -3/50*2.0, 2.0      # beta = 1 (positive beta is a puller (negative V2s))

In [5]:
## initialise 
#initial position
r = np.array([-1.5,1.5, 0,0 , 0., 0.])

#initial orientation: 50 deg towards each other
p0=np.array([0., 1., 0.])
p01 = rotation(p0, 0., 0., -25*2*np.pi/360)
p02 = rotation(p0, 0., 0., 25*2*np.pi/360)
p = np.zeros(3*Np)
p[::Np]  = p01
p[1::Np] = p02

# initial slip modes
S = np.zeros(5*Np)
D = np.zeros(3*Np)
for i in range(Np):
    ## S in order: xx, xy, xz, yy, yz
    S[i]       = S0*(3*p[i]*p[i] -1)
    S[i + Np]  = S0*(3*p[i]*p[i+Np])
    S[i + 2*Np]= S0*(3*p[i]*p[i+2*Np])
    S[i + 3*Np]= S0*(3*p[i+Np]*p[i+Np]-1)
    S[i + 4*Np]= S0*(3*p[i+Np]*p[i+2*Np])
    
    D[i]       = D0*p[i]
    D[i+Np]    = D0*p[i+Np]
    D[i+2*Np]  = D0*p[i+2*Np]

In [ ]:
def rhs(rp):
    r = rp[0:3*Np]; p = rp[3*Np:6*Np]
    
    F, v, o = np.zeros(3*Np), np.zeros(3*Np), np.zeros(3*Np)